✅ 토큰 로드 성공: hf_VQiZxYh...


In [12]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 3.0 MB/s eta 0:00:000:00:01


In [13]:
import os
from dotenv import load_dotenv

load_dotenv()
token = os.getenv('HUGGINGFACE_TOKEN')

if token:
    print(f"✅ 토큰 로드 성공: {token[:10]}...")
else:
    print("❌ 토큰을 불러올 수 없습니다.")
    print("   .env 파일 형식을 확인하세요:")
    print("   HUGGINGFACE_TOKEN=hf_xxxxx (공백 없이)")


✅ 토큰 로드 성공: hf_VQiZxYh...


In [10]:
import numpy as np
import pandas as pd
import os
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_percentage_error, r2_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
import pandas_ta as ta
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
import warnings
from binance.client import Client
import numpy as np
import pandas as pd
import os
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score, accuracy_score
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
import pandas_ta as ta
from datetime import datetime, timedelta
import warnings
from dotenv import load_dotenv
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from huggingface_hub import login as hf_login


warnings.filterwarnings('ignore')


####################################################
# 설정
####################################################
TARGET_MACRO_FILE = 'macro_crypto_data.csv'
ONCHAIN_FILE = 'eth_onchain.csv'
NEWS_DIR = "./news_data"
START_TIME = '2017-01-01'
END_TIME = '2025-10-02'
TARGET_COIN = 'ETH'  # 이더리움 타겟
DEVICE = 'GPU' if len(tf.config.list_physical_devices('GPU')) > 0 else 'CPU'
SEQUENCE_LENGTH = 30
PREDICTION_HORIZON = 1


print(f"실행 환경: {DEVICE}")
print(f"TensorFlow 버전: {tf.__version__}")

####################################################

####################################################
# 1. 데이터 로드 함수
####################################################

def parse_date_from_filename(filename):
    """파일명에서 날짜 추출"""
    patterns = [
        r'(\d{4})-(\d{2})-(\d{2})',
        r'(\d{4})(\d{2})(\d{2})',
        r'(\d{2})-(\d{2})-(\d{4})',
        r'(\d{2})(\d{2})(\d{4})'
    ]
    basename = os.path.basename(filename)
    for pattern in patterns:
        match = re.search(pattern, basename)
        if match:
            try:
                if len(match.group(1)) == 4:
                    year, month, day = match.groups()
                else:
                    day, month, year = match.groups()
                return pd.to_datetime(f"{year}-{month}-{day}")
            except:
                continue
    return None

def load_all_news_data(root_dir):
    """뉴스 데이터 로드 및 일간 집계"""
    print("\n=== 뉴스 데이터 로드 ===")
    
    all_data = []
    
    if not os.path.exists(root_dir):
        print(f"경고: {root_dir} 디렉토리가 존재하지 않습니다. 더미 데이터를 생성합니다.")
        dates = pd.date_range(START_TIME, END_TIME, freq='D')
        return pd.DataFrame({
            'date': dates,
            'news': ['test news'] * len(dates),
            'label': np.random.choice([1, 0, -1], len(dates))
        })
    
    csv_files = sorted([f for f in os.listdir(root_dir) if f.endswith('.csv')])
    print(f"발견된 뉴스 파일: {len(csv_files)}개")
    
    for filename in csv_files:
        filepath = os.path.join(root_dir, filename)
        file_date = parse_date_from_filename(filename)
        
        # 여러 인코딩 시도
        for enc in ['utf-8', 'cp949', 'latin1']:
            try:
                df = pd.read_csv(filepath, encoding=enc)
                break
            except Exception:
                continue
        else:
            print(f"  건너뜀: {filename} (인코딩 실패)")
            continue
        
        # 날짜 컬럼 처리
        if 'date' not in df.columns:
            df['date'] = file_date
        else:
            df['date'] = pd.to_datetime(df['date'], errors='coerce')
            if file_date is not None:
                df['date'] = df['date'].fillna(file_date)
        
        # label 컬럼 확인
        if 'label' not in df.columns:
            print(f"  경고: {filename}에 'label' 컬럼이 없습니다. 0으로 채웁니다.")
            df['label'] = 0
        
        # 필요한 컬럼만 선택
        if 'news' in df.columns:
            df = df[['date', 'news', 'label']]
        else:
            df = df[['date', 'label']]
        
        all_data.append(df)
    
    if len(all_data) == 0:
        print("경고: 유효한 뉴스 파일을 찾을 수 없습니다. 더미 데이터를 생성합니다.")
        dates = pd.date_range(START_TIME, END_TIME, freq='D')
        return pd.DataFrame({
            'date': dates,
            'news': ['test news'] * len(dates),
            'label': np.random.choice([1, 0, -1], len(dates))
        })
    
    # 데이터 결합
    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df['date'] = pd.to_datetime(combined_df['date'], errors='coerce').dt.normalize()
    
    # 일간 감성 스코어 집계 (평균)
    news_agg = combined_df.groupby('date').agg({
        'label': 'mean',  # 감성 스코어 평균
    }).reset_index()
    news_agg.columns = ['date', 'sentiment_score']
    
    # 추가 통계
    news_count = combined_df.groupby('date').size().reset_index(name='news_count')
    news_agg = news_agg.merge(news_count, on='date', how='left')
    
    print(f" 뉴스 데이터 로드 완료: {len(news_agg)}일치 데이터")
    return news_agg





def load_macro_data(filepath):
    """거시경제 + 암호화폐 가격 데이터 로드"""
    print("\n=== Macro 데이터 로드 ===")
    
    df = pd.read_csv(filepath, parse_dates=['Date'])
    df['Date'] = pd.to_datetime(df['Date']).dt.tz_localize(None).dt.normalize()
    df = df.set_index('Date').sort_index()
    
    # 타겟 코인(ETH) 컬럼 확인
    eth_cols = [col for col in df.columns if col.startswith('ETH_')]
    print(f"발견된 ETH 컬럼: {eth_cols}")
    
    print(f"✓ Macro 데이터 로드 완료: {df.shape}")
    print(f"  기간: {df.index.min()} ~ {df.index.max()}")
    print(f"  컬럼 수: {len(df.columns)}")
    
    return df




def load_onchain_data(filepath):
    """온체인 데이터 로드"""
    print("\n=== 온체인 데이터 로드 ===")
    
    df = pd.read_csv(filepath, parse_dates=['date'])
    df['date'] = pd.to_datetime(df['date']).dt.tz_localize(None).dt.normalize()
    df = df.set_index('date').sort_index()
    
    # 컬럼명에 'onchain_' 접두사 추가 (충돌 방지)
    df.columns = ['onchain_' + col for col in df.columns]
    
    print(f"✓ 온체인 데이터 로드 완료: {df.shape}")
    print(f"  기간: {df.index.min()} ~ {df.index.max()}")
    
    return df

실행 환경: GPU
TensorFlow 버전: 2.15.0


In [5]:
####################################################
# 2. 기술적 지표 계산
####################################################

def calculate_technical_indicators(df, target_coin='ETH'):
    """
    pandas_ta를 사용하여 기술적 지표 계산
    """
    print(f"\n=== {target_coin} 기술적 지표 계산 ===")
    
    df = df.copy()
    
    # 타겟 코인의 OHLCV 컬럼 추출
    prefix = f"{target_coin}_"
    open_col = f"{prefix}Open"
    high_col = f"{prefix}High"
    low_col = f"{prefix}Low"
    close_col = f"{prefix}Close"
    volume_col = f"{prefix}Volume"
    
    # 필수 컬럼 확인
    required_cols = [open_col, high_col, low_col, close_col, volume_col]
    missing_cols = [col for col in required_cols if col not in df.columns]
    
    if missing_cols:
        print(f"경고: 필수 컬럼 누락: {missing_cols}")
        return df
    
    # pandas_ta를 위한 임시 DataFrame 생성 (소문자 컬럼명 사용)
    temp_df = pd.DataFrame({
        'open': df[open_col],
        'high': df[high_col],
        'low': df[low_col],
        'close': df[close_col],
        'volume': df[volume_col]
    })
    
    print("기술적 지표 계산 중...")
    
    try:
        # 1. Momentum Indicators
        print("  - Momentum 지표...")
        temp_df.ta.rsi(length=14, append=True)
        temp_df.ta.rsi(length=7, append=True)
        temp_df.ta.rsi(length=21, append=True)
        temp_df.ta.macd(fast=12, slow=26, signal=9, append=True)
        temp_df.ta.stoch(k=14, d=3, append=True)
        temp_df.ta.roc(length=10, append=True)
        temp_df.ta.roc(length=20, append=True)
        temp_df.ta.mom(length=10, append=True)
        temp_df.ta.cci(length=20, append=True)
        temp_df.ta.willr(length=14, append=True)
        
        # 2. Volatility Indicators
        print("  - Volatility 지표...")
        temp_df.ta.bbands(length=20, std=2, append=True)
        temp_df.ta.atr(length=14, append=True)
        temp_df.ta.natr(length=14, append=True)
        temp_df.ta.kc(length=20, append=True)
        
        # 3. Trend Indicators
        print("  - Trend 지표...")
        for length in [5, 10, 20, 50, 200]:
            temp_df.ta.sma(length=length, append=True)
            temp_df.ta.ema(length=length, append=True)
        
        temp_df.ta.adx(length=14, append=True)
        temp_df.ta.aroon(length=25, append=True)
        
        # 4. Volume Indicators
        print("  - Volume 지표...")
        temp_df.ta.obv(append=True)
        temp_df.ta.ad(append=True)
        temp_df.ta.cmf(length=20, append=True)
        temp_df.ta.mfi(length=14, append=True)
        
        # 5. Custom Features
        print("  - Custom 지표...")
        
        # SMA/EMA 차이
        if 'SMA_50' in temp_df.columns and 'SMA_200' in temp_df.columns:
            temp_df['SMA_diff_50_200'] = temp_df['SMA_50'] - temp_df['SMA_200']
        
        if 'EMA_50' in temp_df.columns and 'EMA_200' in temp_df.columns:
            temp_df['EMA_diff_50_200'] = temp_df['EMA_50'] - temp_df['EMA_200']
        
        # Bollinger Band %B
        if 'BBL_20_2.0' in temp_df.columns and 'BBU_20_2.0' in temp_df.columns:
            bb_width = temp_df['BBU_20_2.0'] - temp_df['BBL_20_2.0']
            bb_width = bb_width.replace(0, np.nan)  # 0으로 나누기 방지
            temp_df['BB_percentB'] = (temp_df['close'] - temp_df['BBL_20_2.0']) / bb_width
        
        # Price Rate of Change
        temp_df['price_roc_1d'] = temp_df['close'].pct_change(1)
        temp_df['price_roc_7d'] = temp_df['close'].pct_change(7)
        temp_df['price_roc_30d'] = temp_df['close'].pct_change(30)
        
        # Volume Rate of Change
        temp_df['volume_roc_1d'] = temp_df['volume'].pct_change(1)
        temp_df['volume_roc_7d'] = temp_df['volume'].pct_change(7)
        
        # High-Low Range
        temp_df['hl_range'] = (temp_df['high'] - temp_df['low']) / temp_df['close']
        
        # Close position within range
        range_val = temp_df['high'] - temp_df['low']
        range_val = range_val.replace(0, np.nan)
        temp_df['close_position'] = (temp_df['close'] - temp_df['low']) / range_val
        
        # Moving Average Convergence
        if 'SMA_20' in temp_df.columns:
            temp_df['price_to_sma20'] = temp_df['close'] / temp_df['SMA_20'] - 1
        
        if 'EMA_20' in temp_df.columns:
            temp_df['price_to_ema20'] = temp_df['close'] / temp_df['EMA_20'] - 1
        
        # Volume trend
        if 'SMA_20' in temp_df.columns:
            volume_sma = temp_df['volume'].rolling(window=20).mean()
            temp_df['volume_trend'] = temp_df['volume'] / volume_sma
        
    except Exception as e:
        print(f"  경고: 일부 지표 계산 실패: {e}")
    
    # 원본 OHLCV 컬럼 제거 (중복 방지)
    tech_indicators = temp_df.drop(columns=['open', 'high', 'low', 'close', 'volume'], errors='ignore')
    
    # 컬럼명에 접두사 추가
    tech_indicators.columns = [f'tech_{col}' for col in tech_indicators.columns]
    
    # 무한대 값 처리
    tech_indicators = tech_indicators.replace([np.inf, -np.inf], np.nan)
    
    # 원본 데이터에 병합
    df = pd.concat([df, tech_indicators], axis=1)
    
    print(f"✓ 기술적 지표 계산 완료: {len(tech_indicators.columns)}개 지표 추가")
    
    return df


In [6]:
####################################################
# 3. 거시경제 지표 추가 (선택적)
####################################################

def add_macro_indicators(df):
    """
    yfinance와 fredapi를 통해 거시경제 지표 추가
    """
    print("\n=== 거시경제 지표 추가 (선택적) ===")
    
    try:
        import yfinance as yf
        
        start_date = df.index.min().strftime('%Y-%m-%d')
        end_date = df.index.max().strftime('%Y-%m-%d')
        
        macro_data = pd.DataFrame(index=df.index)
        
        # DXY (US Dollar Index)
        try:
            dxy = yf.download('DX-Y.NYB', start=start_date, end=end_date, progress=False)['Close']
            macro_data['macro_DXY'] = dxy
            print("  ✓ DXY 추가")
        except Exception as e:
            print(f"  ✗ DXY 실패: {e}")
        
        # Gold
        try:
            gold = yf.download('GC=F', start=start_date, end=end_date, progress=False)['Close']
            macro_data['macro_GOLD'] = gold
            print("  ✓ GOLD 추가")
        except Exception as e:
            print(f"  ✗ GOLD 실패: {e}")
        
        # VIX
        try:
            vix = yf.download('^VIX', start=start_date, end=end_date, progress=False)['Close']
            macro_data['macro_VIX'] = vix
            print("  ✓ VIX 추가")
        except Exception as e:
            print(f"  ✗ VIX 실패: {e}")
        
        # S&P 500
        try:
            sp500 = yf.download('^GSPC', start=start_date, end=end_date, progress=False)['Close']
            macro_data['macro_SP500'] = sp500
            print("  ✓ S&P500 추가")
        except Exception as e:
            print(f"  ✗ S&P500 실패: {e}")
        
        # Forward fill missing values
        macro_data = macro_data.ffill()
        
        # 원본 데이터에 병합
        df = df.join(macro_data, how='left')
        
        print(f"✓ 거시경제 지표 추가 완료: {len([col for col in df.columns if col.startswith('macro_')])}개")
        
    except ImportError:
        print("  경고: yfinance가 설치되지 않았습니다. 거시경제 지표를 건너뜁니다.")
        print("  설치: pip install yfinance")
    
    return df

####################################################
# 4. 선물 시장 데이터 추가 (선택적)
####################################################

def add_futures_data(df, symbol='ETHUSDT'):
    """
    Binance API를 통해 선물 시장 데이터 추가
    """
    print("\n=== 선물 시장 데이터 추가 (선택적) ===")
    
    try:
        from binance.client import Client
        
        client = Client("", "")  # API Key 없이 공개 데이터 접근
        
        start_date = df.index.min()
        end_date = df.index.max()
        
        print("  Funding Rate 수집 중...")
        funding_rates = []
        
        start_ts = int(pd.Timestamp(start_date).timestamp() * 1000)
        end_ts = int(pd.Timestamp(end_date).timestamp() * 1000)
        
        current_ts = start_ts
        chunk_count = 0
        
        while current_ts < end_ts and chunk_count < 10:  # 최대 10 청크
            try:
                rates = client.futures_funding_rate(
                    symbol=symbol,
                    startTime=current_ts,
                    limit=1000
                )
                if not rates:
                    break
                funding_rates.extend(rates)
                current_ts = rates[-1]['fundingTime'] + 1
                chunk_count += 1
            except Exception as e:
                print(f"    청크 수집 실패: {e}")
                break
        
        if funding_rates:
            df_funding = pd.DataFrame(funding_rates)
            df_funding['fundingTime'] = pd.to_datetime(df_funding['fundingTime'], unit='ms')
            df_funding.set_index('fundingTime', inplace=True)
            df_funding['fundingRate'] = df_funding['fundingRate'].astype(float)
            
            # 일간 평균으로 리샘플링
            funding_daily = df_funding['fundingRate'].resample('D').mean()
            df['futures_funding_rate'] = funding_daily
            
            print(f"  ✓ Funding Rate 추가: {len(df_funding)} 개 데이터")
        else:
            print("  ✗ Funding Rate 데이터 없음")
    
    except ImportError:
        print("  경고: python-binance가 설치되지 않았습니다. 선물 데이터를 건너뜁니다.")
        print("  설치: pip install python-binance")
    except Exception as e:
        print(f"  ✗ 선물 데이터 수집 실패: {e}")
    
    return df

In [1]:
import numpy as np
import pandas as pd
import os
import re
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score, accuracy_score
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.optimizers import Adam
import pandas_ta as ta
from datetime import datetime, timedelta
import warnings
from dotenv import load_dotenv

warnings.filterwarnings('ignore')

TARGET_MACRO_FILE = 'macro_crypto_data.csv'
ONCHAIN_FILE = 'eth_onchain.csv'
NEWS_DIR = "./news_data"
START_TIME = '2022-09-15'
END_TIME = '2025-10-02'
TARGET_COIN = 'ETH'
SEQUENCE_LENGTH = 10
PREDICTION_HORIZON = 1
TOP_K_FEATURES = 40

load_dotenv()

def parse_date_from_filename(filename):
    patterns = [
        r'(\d{4})-(\d{2})-(\d{2})',
        r'(\d{4})(\d{2})(\d{2})',
        r'(\d{2})-(\d{2})-(\d{4})',
        r'(\d{2})(\d{2})(\d{4})'
    ]
    basename = os.path.basename(filename)
    for pattern in patterns:
        match = re.search(pattern, basename)
        if match:
            try:
                if len(match.group(1)) == 4:
                    year, month, day = match.groups()
                else:
                    day, month, year = match.groups()
                return pd.to_datetime(f"{year}-{month}-{day}")
            except:
                continue
    return None

def load_all_news_data(root_dir):
    all_data = []
    
    if not os.path.exists(root_dir):
        dates = pd.date_range(START_TIME, END_TIME, freq='D')
        return pd.DataFrame({
            'date': dates,
            'sentiment_score': np.zeros(len(dates)),
            'news_count': np.ones(len(dates))
        })
    
    csv_files = sorted([f for f in os.listdir(root_dir) if f.endswith('.csv')])
    
    for filename in csv_files:
        filepath = os.path.join(root_dir, filename)
        file_date = parse_date_from_filename(filename)
        
        for enc in ['utf-8', 'cp949', 'latin1']:
            try:
                df = pd.read_csv(filepath, encoding=enc)
                break
            except Exception:
                continue
        else:
            continue
        
        if 'date' not in df.columns:
            df['date'] = file_date
        else:
            df['date'] = pd.to_datetime(df['date'], errors='coerce')
            if file_date is not None:
                df['date'] = df['date'].fillna(file_date)
        
        if 'label' not in df.columns:
            df['label'] = 0
        
        all_data.append(df[['date', 'label']])
    
    if len(all_data) == 0:
        dates = pd.date_range(START_TIME, END_TIME, freq='D')
        return pd.DataFrame({
            'date': dates,
            'sentiment_score': np.zeros(len(dates)),
            'news_count': np.ones(len(dates))
        })
    
    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df['date'] = pd.to_datetime(combined_df['date'], errors='coerce').dt.normalize()
    
    news_agg = combined_df.groupby('date').agg({
        'label': 'mean',
    }).reset_index()
    news_agg.columns = ['date', 'sentiment_score']
    
    news_count = combined_df.groupby('date').size().reset_index(name='news_count')
    news_agg = news_agg.merge(news_count, on='date', how='left')
    
    return news_agg

def load_macro_data(filepath):
    df = pd.read_csv(filepath, parse_dates=['Date'])
    df['Date'] = pd.to_datetime(df['Date']).dt.tz_localize(None).dt.normalize()
    df = df.set_index('Date').sort_index()
    df = df[df.index >= START_TIME]
    return df

def load_onchain_data(filepath):
    if not os.path.exists(filepath):
        return None
    df = pd.read_csv(filepath, parse_dates=['date'])
    df['date'] = pd.to_datetime(df['date']).dt.tz_localize(None).dt.normalize()
    df = df.set_index('date').sort_index()
    df = df[df.index >= START_TIME]
    df.columns = ['onchain_' + col for col in df.columns]
    return df

def calculate_directional_features(df, target_coin='ETH'):
    df = df.copy()
    
    prefix = f"{target_coin}_"
    close_col = f"{prefix}Close"
    volume_col = f"{prefix}Volume"
    high_col = f"{prefix}High"
    low_col = f"{prefix}Low"
    
    if close_col not in df.columns:
        return df
    
    df[f'{prefix}returns_1d'] = df[close_col].pct_change(1)
    df[f'{prefix}returns_3d'] = df[close_col].pct_change(3)
    df[f'{prefix}returns_7d'] = df[close_col].pct_change(7)
    
    df[f'{prefix}momentum_5'] = df[close_col] - df[close_col].shift(5)
    df[f'{prefix}momentum_10'] = df[close_col] - df[close_col].shift(10)
    
    for period in [5, 10, 20]:
        sma = df[close_col].rolling(period).mean()
        df[f'{prefix}sma_{period}'] = sma
        df[f'{prefix}price_to_sma_{period}'] = (df[close_col] / sma - 1) * 100
        df[f'{prefix}sma_{period}_slope'] = sma.diff(1)
    
    ema_fast = df[close_col].ewm(span=12).mean()
    ema_slow = df[close_col].ewm(span=26).mean()
    df[f'{prefix}macd'] = ema_fast - ema_slow
    df[f'{prefix}macd_signal'] = df[f'{prefix}macd'].ewm(span=9).mean()
    df[f'{prefix}macd_diff'] = df[f'{prefix}macd'] - df[f'{prefix}macd_signal']
    
    delta = df[close_col].diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = -delta.where(delta < 0, 0).rolling(14).mean()
    rs = gain / loss
    df[f'{prefix}rsi'] = 100 - (100 / (1 + rs))
    df[f'{prefix}rsi_oversold'] = (df[f'{prefix}rsi'] < 30).astype(int)
    df[f'{prefix}rsi_overbought'] = (df[f'{prefix}rsi'] > 70).astype(int)
    
    df[f'{prefix}volatility_5'] = df[close_col].pct_change().rolling(5).std()
    df[f'{prefix}volatility_10'] = df[close_col].pct_change().rolling(10).std()
    
    if volume_col in df.columns:
        df[f'{prefix}volume_change'] = df[volume_col].pct_change(1)
        df[f'{prefix}volume_sma_10'] = df[volume_col].rolling(10).mean()
        df[f'{prefix}volume_ratio'] = df[volume_col] / df[f'{prefix}volume_sma_10']
        
        price_change = df[close_col].diff()
        df[f'{prefix}obv'] = (np.sign(price_change) * df[volume_col]).fillna(0).cumsum()
        df[f'{prefix}obv_slope'] = df[f'{prefix}obv'].diff(1)
    
    if high_col in df.columns and low_col in df.columns:
        df[f'{prefix}atr'] = ((df[high_col] - df[low_col]) / df[close_col]).rolling(14).mean()
        df[f'{prefix}price_position'] = (df[close_col] - df[low_col]) / (df[high_col] - df[low_col])
    
    for i in [1, 2, 3, 5, 7]:
        df[f'{prefix}higher_than_{i}d_ago'] = (df[close_col] > df[close_col].shift(i)).astype(int)
    
    return df

def merge_all_data():
    macro_df = load_macro_data(TARGET_MACRO_FILE)
    
    onchain_df = load_onchain_data(ONCHAIN_FILE)
    if onchain_df is not None:
        macro_df = macro_df.join(onchain_df, how='left')
    
    news_df = load_all_news_data(NEWS_DIR)
    news_df = news_df.set_index('date')
    macro_df = macro_df.join(news_df, how='left')
    
    macro_df = calculate_directional_features(macro_df, TARGET_COIN)
    
    macro_df = macro_df.replace([np.inf, -np.inf], np.nan)
    macro_df = macro_df.ffill().bfill()
    
    return macro_df

def remove_correlated_features(X, threshold=0.90):
    corr_matrix = np.corrcoef(X.T)
    corr_matrix = np.abs(corr_matrix)
    
    upper_triangle = np.triu(corr_matrix, k=1)
    
    to_remove = set()
    for i in range(upper_triangle.shape[0]):
        for j in range(i+1, upper_triangle.shape[1]):
            if upper_triangle[i, j] > threshold:
                to_remove.add(j)
    
    keep_indices = [i for i in range(X.shape[1]) if i not in to_remove]
    return keep_indices

def select_features(X, y, feature_names, k=40):
    keep_indices = remove_correlated_features(X, threshold=0.90)
    X_filtered = X[:, keep_indices]
    feature_names_filtered = [feature_names[i] for i in keep_indices]
    
    selector = SelectKBest(score_func=mutual_info_regression, k=min(k, X_filtered.shape[1]))
    selector.fit(X_filtered, y)
    selected_mask = selector.get_support()
    
    final_indices = [keep_indices[i] for i, selected in enumerate(selected_mask) if selected]
    final_features = [feature_names[i] for i in final_indices]
    
    return final_indices, final_features

def create_directional_sequences(data, target, seq_length=10):
    X, y_price, y_direction = [], [], []
    for i in range(len(data) - seq_length - 1):
        X.append(data[i:i+seq_length])
        
        current_price = target[i+seq_length-1]
        next_price = target[i+seq_length]
        
        y_price.append(next_price)
        y_direction.append(1 if next_price > current_price else 0)
    
    return np.array(X), np.array(y_price), np.array(y_direction)

def build_attention_model(input_shape):
    inputs = Input(shape=input_shape)
    
    x = layers.LSTM(128, return_sequences=True)(inputs)
    x = layers.Dropout(0.3)(x)
    
    attention = layers.Dense(1, activation='tanh')(x)
    attention = layers.Flatten()(attention)
    attention = layers.Activation('softmax')(attention)
    attention = layers.RepeatVector(128)(attention)
    attention = layers.Permute([2, 1])(attention)
    
    x_attended = layers.multiply([x, attention])
    x_attended = layers.Lambda(lambda xin: tf.reduce_sum(xin, axis=1))(x_attended)
    
    x = layers.Dense(64, activation='relu')(x_attended)
    x = layers.Dropout(0.3)(x)
    
    direction_branch = layers.Dense(32, activation='relu')(x)
    direction_branch = layers.Dropout(0.2)(direction_branch)
    direction_output = layers.Dense(1, activation='sigmoid', name='direction')(direction_branch)
    
    price_branch = layers.Dense(32, activation='relu')(x)
    price_branch = layers.Dropout(0.2)(price_branch)
    price_output = layers.Dense(1, name='price')(price_branch)
    
    model = Model(inputs=inputs, outputs=[price_output, direction_output])
    
    return model

def calculate_trading_performance(y_true_price, y_pred_price, y_true_dir, y_pred_dir_prob):
    y_pred_dir = (y_pred_dir_prob > 0.5).astype(int).flatten()
    y_true_dir = y_true_dir.flatten()
    
    direction_acc = accuracy_score(y_true_dir, y_pred_dir)
    
    returns_true = np.diff(y_true_price.flatten()) / y_true_price.flatten()[:-1]
    
    signals = y_pred_dir[:-1]
    signals = np.where(signals == 0, -1, 1)
    
    strategy_returns = signals * returns_true
    
    cumulative_returns = np.cumprod(1 + strategy_returns) - 1
    total_return = cumulative_returns[-1] if len(cumulative_returns) > 0 else 0
    
    sharpe = np.mean(strategy_returns) / (np.std(strategy_returns) + 1e-10) * np.sqrt(252)
    
    cumulative_max = np.maximum.accumulate(1 + cumulative_returns)
    drawdown = (cumulative_max - (1 + cumulative_returns)) / cumulative_max
    max_dd = np.max(drawdown) if len(drawdown) > 0 else 0
    
    win_rate = np.mean(strategy_returns > 0) if len(strategy_returns) > 0 else 0
    
    return {
        'direction_accuracy': direction_acc,
        'total_return': total_return,
        'sharpe_ratio': sharpe,
        'max_drawdown': max_dd,
        'win_rate': win_rate
    }

def main():
    print("=" * 70)
    print("Enhanced Directional Prediction Model")
    print("=" * 70)
    
    print(f"\nPeriod: {START_TIME} to {END_TIME}")
    
    print("\nStep 1: Loading Data")
    df = merge_all_data()
    print(f"Samples: {len(df)}")
    
    target_col = f'{TARGET_COIN}_Close'
    if target_col not in df.columns:
        print(f"Error: {target_col} not found")
        return
    
    print("\nStep 2: Feature Selection")
    feature_cols = [col for col in df.columns if col not in [target_col] and not col.endswith('_Close')]
    
    df_clean = df[[target_col] + feature_cols].dropna()
    print(f"Clean samples: {len(df_clean)}")
    print(f"Initial features: {len(feature_cols)}")
    
    selected_indices, selected_features = select_features(
        df_clean[feature_cols].values,
        df_clean[target_col].values,
        feature_cols,
        k=TOP_K_FEATURES
    )
    print(f"Selected features: {len(selected_features)}")
    
    print("\nTop features:")
    for i, feat in enumerate(selected_features[:15], 1):
        print(f"{i:2d}. {feat}")
    
    print("\nStep 3: Scaling")
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    
    X_scaled = scaler_X.fit_transform(df_clean[selected_features])
    y_values = df_clean[target_col].values.reshape(-1, 1)
    y_scaled = scaler_y.fit_transform(y_values).flatten()
    
    print("\nStep 4: Creating Sequences")
    X_seq, y_price, y_direction = create_directional_sequences(X_scaled, y_scaled, SEQUENCE_LENGTH)
    print(f"Sequences: {X_seq.shape}")
    print(f"Direction balance: {np.mean(y_direction):.2%} up")
    
    print("\nStep 5: Train/Val/Test Split")
    train_size = int(len(X_seq) * 0.7)
    val_size = int(len(X_seq) * 0.15)
    
    X_train = X_seq[:train_size]
    y_train_price = y_price[:train_size]
    y_train_dir = y_direction[:train_size]
    
    X_val = X_seq[train_size:train_size+val_size]
    y_val_price = y_price[train_size:train_size+val_size]
    y_val_dir = y_direction[train_size:train_size+val_size]
    
    X_test = X_seq[train_size+val_size:]
    y_test_price = y_price[train_size+val_size:]
    y_test_dir = y_direction[train_size+val_size:]
    
    print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")
    
    print("\nStep 6: Building Model with Attention")
    model = build_attention_model((X_train.shape[1], X_train.shape[2]))
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss={
            'price': 'huber',
            'direction': 'binary_crossentropy'
        },
        loss_weights={
            'price': 0.6,
            'direction': 0.4
        },
        metrics={
            'price': ['mae'],
            'direction': ['accuracy']
        }
    )
    
    print("\nStep 7: Training")
    history = model.fit(
        X_train,
        {'price': y_train_price, 'direction': y_train_dir},
        validation_data=(
            X_val,
            {'price': y_val_price, 'direction': y_val_dir}
        ),
        epochs=200,
        batch_size=32,
        verbose=1,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(
                patience=40,
                restore_best_weights=True,
                monitor='val_direction_accuracy',
                mode='max'
            ),
            tf.keras.callbacks.ReduceLROnPlateau(
                factor=0.5,
                patience=20,
                min_lr=1e-7,
                monitor='val_direction_accuracy',
                mode='max'
            )
        ]
    )
    
    print("\nStep 8: Evaluation")
    y_pred_price, y_pred_dir = model.predict(X_test)
    
    y_test_price_original = scaler_y.inverse_transform(y_test_price.reshape(-1, 1))
    y_pred_price_original = scaler_y.inverse_transform(y_pred_price)
    
    mse = mean_squared_error(y_test_price_original, y_pred_price_original)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(y_test_price_original, y_pred_price_original)
    r2 = r2_score(y_test_price_original, y_pred_price_original)
    
    print(f"\nPrice Metrics:")
    print(f"RMSE: ${rmse:.2f}")
    print(f"MAPE: {mape*100:.2f}%")
    print(f"R2: {r2:.4f}")
    
    trading_perf = calculate_trading_performance(
        y_test_price_original,
        y_pred_price_original,
        y_test_dir,
        y_pred_dir
    )
    
    print(f"\nTrading Performance:")
    print(f"Direction Accuracy: {trading_perf['direction_accuracy']*100:.2f}%")
    print(f"Total Return: {trading_perf['total_return']*100:.2f}%")
    print(f"Sharpe Ratio: {trading_perf['sharpe_ratio']:.2f}")
    print(f"Max Drawdown: {trading_perf['max_drawdown']*100:.2f}%")
    print(f"Win Rate: {trading_perf['win_rate']*100:.2f}%")
    
    print("\n" + "=" * 70)
    print("Complete")
    print("=" * 70)
    
    return model, scaler_X, scaler_y, selected_features, df_clean

if __name__ == "__main__":
    model, scaler_X, scaler_y, selected_features, df_clean = main()


2025-10-04 15:47:32.275910: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-04 15:47:32.275954: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-04 15:47:32.277181: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-04 15:47:32.283868: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-04 15:47:33.031437: W tensorflow/comp

Enhanced Directional Prediction Model

Period: 2022-09-15 to 2025-10-02

Step 1: Loading Data
Samples: 1115

Step 2: Feature Selection
Clean samples: 1115
Initial features: 86
Selected features: 40

Top features:
 1. BTC_Open
 2. BTC_Volume
 3. BNB_Volume
 4. XRP_Open
 5. XRP_Volume
 6. SOL_Volume
 7. ADA_Open
 8. ADA_Volume
 9. DOGE_Open
10. DOGE_Volume
11. AVAX_Open
12. AVAX_Volume
13. DOT_Open
14. DOT_Volume
15. MATIC_Open

Step 3: Scaling

Step 4: Creating Sequences
Sequences: (1104, 10, 40)
Direction balance: 50.91% up

Step 5: Train/Val/Test Split
Train: 772, Val: 165, Test: 167

Step 6: Building Model with Attention


2025-10-04 15:47:46.307079: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46689 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:1d:00.0, compute capability: 8.6



Step 7: Training
Epoch 1/200


2025-10-04 15:47:51.726765: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2025-10-04 15:47:53.558460: I external/local_xla/xla/service/service.cc:168] XLA service 0x2bc7cec0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-10-04 15:47:53.558493: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2025-10-04 15:47:53.565650: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1759560473.719739  180510 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


25/25 [==============================] - 8s 27ms/step - loss: 0.3549 - price_loss: 0.1252 - direction_loss: 0.6993 - price_mae: 0.3770 - direction_accuracy: 0.4870 - val_loss: 0.3515 - val_price_loss: 0.1195 - val_direction_loss: 0.6995 - val_price_mae: 0.4271 - val_direction_accuracy: 0.4970 - lr: 0.0010
Epoch 2/200
25/25 [==============================] - 0s 7ms/step - loss: 0.3101 - price_loss: 0.0511 - direction_loss: 0.6986 - price_mae: 0.2467 - direction_accuracy: 0.4780 - val_loss: 0.3656 - val_price_loss: 0.1414 - val_direction_loss: 0.7018 - val_price_mae: 0.4599 - val_direction_accuracy: 0.4909 - lr: 0.0010
Epoch 3/200
25/25 [==============================] - 0s 7ms/step - loss: 0.3004 - price_loss: 0.0407 - direction_loss: 0.6899 - price_mae: 0.2197 - direction_accuracy: 0.5337 - val_loss: 0.3506 - val_price_loss: 0.1195 - val_direction_loss: 0.6972 - val_price_mae: 0.4315 - val_direction_accuracy: 0.4667 - lr: 0.0010
Epoch 4/200
25/25 [==============================] - 0s 7

Epoch 27/200
25/25 [==============================] - 0s 7ms/step - loss: 0.2801 - price_loss: 0.0209 - direction_loss: 0.6689 - price_mae: 0.1572 - direction_accuracy: 0.5907 - val_loss: 0.3474 - val_price_loss: 0.0970 - val_direction_loss: 0.7228 - val_price_mae: 0.3726 - val_direction_accuracy: 0.4606 - lr: 5.0000e-04
Epoch 28/200
25/25 [==============================] - 0s 7ms/step - loss: 0.2807 - price_loss: 0.0214 - direction_loss: 0.6697 - price_mae: 0.1528 - direction_accuracy: 0.5933 - val_loss: 0.3553 - val_price_loss: 0.1067 - val_direction_loss: 0.7282 - val_price_mae: 0.3935 - val_direction_accuracy: 0.4727 - lr: 5.0000e-04
Epoch 29/200
25/25 [==============================] - 0s 7ms/step - loss: 0.2796 - price_loss: 0.0236 - direction_loss: 0.6636 - price_mae: 0.1618 - direction_accuracy: 0.6075 - val_loss: 0.3469 - val_price_loss: 0.0895 - val_direction_loss: 0.7331 - val_price_mae: 0.3558 - val_direction_accuracy: 0.4545 - lr: 5.0000e-04
Epoch 30/200
25/25 [===========